# Data Collection and Cleaning
This notebook is tasked with sanitizing and processing various pieces of information about election and registration data, for later use by the "Data Exploration" and "Machine Learning" notebooks.

### Data Files

#### Understanding file paths

All of the files consumed by this notebook can be found in `data/raw/`

These follow a form similar to: `c079_g16_sov_data_by_g16_srprec.csv` 
In this instance, '079' refers to the county (San Luis Obispo), while 'g16' refers to the election type and year (2016 general election). Finally, 'by_g16_srprec' refers to the type of precinct coding.

#### SOV Files

*e.g.* `c083_g18_sov_data_by_g18_srprec.csv`

"Statement of Vote" files. These files contain information particular to the 

#### VOTE Files

*e.g.* `c083_g18_voters_by_g18_srprec.csv`

These files contain registration information for those who participated/voted in a particular election.

#### REG Files

*e.g.* `c083_g18_registration_by_g18_srprec.csv`

These files contain registration information for *all* registered voters in the county.


## Imports

In [15]:
import pandas as pd 

## Data Cleaning

In [17]:
sov_path = "data/raw/c0{county}_g{year}_sov_data_by_g{year}_srprec.csv"
sor_path = "data/raw/c0{county}_g{year}_voters_by_g{year}_srprec.csv"
reg_path = "data/raw/c0{county}_g{year}_registration_by_g{year}_srprec.csv"

sov_san_luis = []
sov_santa_barbara = []
sor_san_luis = []
sor_santa_barbara = []
reg_san_luis = []
reg_santa_barbara = []

df_sov_sb_combined = None
df_sov_sl_combined = None

df_election_totals = pd.DataFrame()
df_registration_voted_totals = pd.DataFrame()
df_registration_totals = pd.DataFrame()

# Iterate over election cycles.
for year in ["12", "14", "16", "18", "20"]:

    # Read sov data for each county.
    df_sov_sb = pd.read_csv(sov_path.format(county="83", year=year))
    df_sov_sl = pd.read_csv(sov_path.format(county="79", year=year))

    # Read sor data for each county.
    df_sor_sb = pd.read_csv(sor_path.format(county="83", year=year))
    df_sor_sl = pd.read_csv(sor_path.format(county="79", year=year))

    # Read registration information for each county.
    df_reg_sb = pd.read_csv(reg_path.format(county="83", year=year))
    df_reg_sl = pd.read_csv(reg_path.format(county="79", year=year))

    # Filter precincts to only those that are in the correct district.
    df_sov_sb = df_sov_sb[df_sov_sb["addist"] == 35]
    df_sov_sl = df_sov_sl[df_sov_sl["addist"] == 35]

    # Make fields for the results of the top-ballot race candidates.
    top_race = "GOV" if year in ["14", "18"] else "PRS"
    df_sov_sb["TOPREP01"] = df_sov_sb["{}REP01".format(top_race)]
    df_sov_sb["TOPDEM01"] = df_sov_sb["{}DEM01".format(top_race)]
    df_sov_sl["TOPREP01"] = df_sov_sl["{}REP01".format(top_race)]
    df_sov_sl["TOPDEM01"] = df_sov_sl["{}DEM01".format(top_race)]


    # SOR files don't contain information about what AD they belong to. 
    # So we need to filter them by the related SOV file. 
    df_sor_sb = df_sor_sb[df_sor_sb["srprec"].isin(df_sov_sb["srprec"])]
    df_sor_sl = df_sor_sl[df_sor_sl["srprec"].isin(df_sor_sl["srprec"])]

    # Likewise, registration files do not include assembly district to filter on.
    df_reg_sb = df_reg_sb[df_reg_sb["srprec"].isin(df_sov_sb["srprec"])]
    df_reg_sl = df_reg_sl[df_reg_sl["srprec"].isin(df_sov_sl["srprec"])]

    # Get totals. We can't use the reported county totals - because they don't
    # necessarily reflect the totals of assembly districts.
    sr_sov_sb_totals = df_sov_sb.iloc[:, 6:].sum()
    sr_sov_sl_totals = df_sov_sl.iloc[:, 6:].sum()
    sr_cycle_totals = sr_sov_sb_totals + sr_sov_sl_totals

    # Add the cycle year to summated Series. 
    sr_year = pd.Series(data=["20" + year], index=["year"])
    sr_cycle_totals = sr_cycle_totals.append(sr_year)

    # Total information about registration for those who voted.
    sr_sor_sb_totals = df_sor_sb.iloc[:, 3:].sum()
    sr_sor_sl_totals = df_sor_sl.iloc[:, 3:].sum()
    sr_reg_vote_totals = sr_sor_sb_totals + sr_sor_sl_totals

    sr_reg_vote_totals = sr_reg_vote_totals.append(sr_year)

    # Total registration information for all voters in the district.
    sr_reg_sb_totals = df_reg_sb.iloc[:, 3:].sum()
    sr_reg_sl_totals = df_reg_sl.iloc[:, 3:].sum()
    sr_reg_totals = sr_reg_sb_totals + sr_reg_sl_totals

    sr_reg_totals = sr_reg_totals.append(sr_year)

    # Add summated totals to election total
    df_election_totals = df_election_totals.append(sr_cycle_totals, ignore_index=True)
    df_registration_voted_totals = df_registration_voted_totals.append(sr_reg_vote_totals, ignore_index=True)
    df_registration_totals = df_registration_totals.append(sr_reg_totals, ignore_index=True)
   
    # Determine the previous cycle - used to suffix columns labels during merge
    prev = str(int(year) - 2)

    # Merge datasets by precinct.  
    df_sov_sb_combined = df_sov_sb if df_sov_sb_combined is None else \
        df_sov_sb_combined.merge(
            right=df_sov_sb, 
            on="srprec", 
            how="inner", 
            suffixes=["_{}".format(prev), "_{}".format(year)])

    df_sov_sl_combined = df_sov_sl if df_sov_sl_combined is None else \
        df_sov_sl_combined.merge(
            right=df_sov_sl, 
            on="srprec", 
            how="inner", 
            suffixes=["_{}".format(prev), "_{}".format(year)])

   

    # Add a year column to the dataset.
    df_sov_sb["year"] = "20{}".format(year)
    df_sov_sl["year"] = "20{}".format(year)    

    # Append to list of dataframes for later work.
    sov_san_luis.append(df_sov_sl)
    sov_santa_barbara.append(df_sov_sb)

    df_reg_sb["year"] = "20{}".format(year)
    df_reg_sl["year"] = "20{}".format(year)

    reg_san_luis.append(df_reg_sl)
    reg_santa_barbara.append(df_reg_sb)

df_election_totals.set_index("year", inplace=True)
df_registration_voted_totals.set_index("year", inplace=True)
df_registration_totals.set_index("year", inplace=True)

# Debug outputs
# df_election_totals
#df_registration_voted_totals
# df_registration_totals - df_registration_voted_totals


## Joining Datasets

In [19]:
df_concat_sb = pd.concat(sov_santa_barbara, axis=0)
df_filtered_sb = df_concat_sb.groupby("srprec").filter(lambda x : len(x) > 2)
# print(df_filtered_sb["year"].unique())
print("Filtered precincts {} vs {} total precincts".format(df_filtered_sb.shape[0], df_concat_sb.shape[0]))

df_concat_slo = pd.concat(sov_san_luis, axis=0)
df_filtered_slo = df_concat_slo.groupby("srprec").filter(lambda x : len(x) > 4)
print("Filtered precincts {} vs {} total precincts".format(df_filtered_slo.shape[0], df_concat_slo.shape[0]))


df_combined_precincts = pd.concat([df_concat_sb, df_concat_slo])
df_precincts_over_time = df_combined_precincts.groupby("srprec").filter(lambda x : len(x) > 4)
print("Filtered precincts {} vs {} total precincts".format(df_precincts_over_time.shape[0], df_combined_precincts.shape[0]))

df_combined_registration = pd.concat(reg_san_luis, axis=0)
df_combined_registration.append(pd.concat(reg_santa_barbara, axis=0))

df_precincts_over_time_with_registration = df_precincts_over_time.merge(
    df_combined_registration, 
    how="inner",
    on=["srprec", "year"]
)

df_precincts_over_time_with_registration


Filtered precincts 171 vs 431 total precincts
Filtered precincts 380 vs 725 total precincts
Filtered precincts 380 vs 1156 total precincts


,county,srprec,addist,cddist,sddist,bedist,TOTREG,DEMREG,REPREG,AIPREG,...,ireg9g,oreg1g,oreg2g,oreg3g,oreg4g,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g
0,40,103,35,24,17,2.0,1304,0,0,0,...,26,17,15,13,2,7,4,5,4,14
1,40,104,35,24,17,2.0,1466,0,0,0,...,21,11,13,11,10,9,3,5,3,10
2,40,105,35,24,17,2.0,668,0,0,0,...,16,12,5,6,1,5,1,0,0,6
3,40,106,35,24,17,2.0,1302,0,0,0,...,14,10,14,9,3,4,2,2,1,3
4,40,107,35,24,17,2.0,533,0,0,0,...,4,7,4,4,3,3,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,40,517,35,24,17,2.0,898,0,0,0,...,30,43,9,7,0,2,0,2,3,7
376,40,518,35,24,17,2.0,2644,0,0,0,...,27,108,25,9,2,8,1,1,1,4
377,40,519,35,24,17,2.0,2812,0,0,0,...,25,119,17,8,2,3,1,3,0,5
378,40,520,35,24,17,2.0,1913,0,0,0,...,8,100,18,5,2,2,1,2,0,1


## Validating Results   

We can perform a basic sanity check on the results of our filtered data, by comparing our own summated results of the assembly races with the certified totals. These totals were retrieved from BallotPedia 

In [21]:

confirmed_totals = { '2020' : [103206, 126579], 
#                     '2018' : [76994, 97749],
                     '2016' : [87168, 105247], 
                     '2014' : [46126, 77452],
                     '2012' : [65500, 103762]}


for yr in confirmed_totals:
    demVotes = df_election_totals.loc[yr]["ASSDEM01"]
    repVotes = df_election_totals.loc[yr]["ASSREP01"]
    demExpected = confirmed_totals[yr][0]
    repExpected = confirmed_totals[yr][1]
    assert demExpected == demVotes, "({}) Data was {} but expected: {}".format(yr, demVotes, demExpected)
    assert repExpected == repVotes, "({}) Data was {} but expected: {}".format(yr, repVotes, repExpected)

Interestingly, and rather frustratingly, it appears that Santa Barbara's 2018 county file does not actually contain information about the 35th's assembly votes. 

*Note: I've actually reached out to StatewideDatabase, and they've confirmed that this is a previously undocumented issue. I've been informed by a developer that this is an issue that will be looked into*


## Exporting Results

In [22]:
df_election_totals.to_csv("data/processed/election_totals.csv")
df_registration_voted_totals.to_csv("data/processed/registration_voted_totals.csv")
df_registration_totals.to_csv("data/processed/registration_totals.csv")
df_precincts_over_time.to_csv("data/processed/precincts_over_time.csv")
df_precincts_over_time_with_registration.to_csv("data/processed/precincts_with_registration.csv")